<a href="https://colab.research.google.com/github/avrymi-asraf/Garden-of-GAN/blob/main/1-Basic-And-Principles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic and Principle

## Installations and Import

In [1]:
import torch
from torch import nn, optim
import torch.utils.data.dataloader as dataloader
from torchvision import datasets, transforms
import plotly.express as px
import pandas as pd
import plotly.figure_factory as ff
from IPython.display import clear_output

In [2]:
from typing import Optional, Callable

## Simple Model

## Gan network
gan network is tow networks that work thogeber to train.
one net is the Generator, the seconde is Discrimnator.
the Discrimnator try to classify the data, and the generator try to make good examples.

In [54]:
# Define the generator network
class Generator(nn.Module):
    def __init__(self, letant_dim, output_dim):
        super().__init__()
        # MLP with dense layers and activations
        self.mlp = nn.Sequential(nn.Linear(letant_dim,int(letant_dim*1.5),),nn.LeakyReLU(),nn.Linear(int(letant_dim*1.5),letant_dim*2),nn.LeakyReLU(),nn.Linear(letant_dim*2,output_dim))

    def forward(self, z):
        return self.mlp(z)


# Define the discriminator network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        # MLP with dense layers and activations
        self.mlp = nn.Sequential(nn.Linear(input_dim,int(input_dim*1.5),),nn.LeakyReLU(),nn.Linear(int(input_dim*1.5),input_dim*2),nn.LeakyReLU(),nn.Linear(input_dim*2,1),)

    def forward(self, x):
        return nn.functional.sigmoid(self.mlp(x))

In [34]:
def gaussians(
    batch_size: int = 64,
    mean: Optional[torch.Tensor] = None,
    var: Optional[torch.Tensor] = None,
) -> Callable[[], torch.Tensor]:
    """creates a function that returns a tensor of gaussian distributed numbers

    Args:
        batch_size (int, optional): batch size. Defaults to 64.
        mean (Optional[torch.Tensor], optional): if determinte the mean or determin randomly. Defaults to None.
        var (Optional[torch.Tensor], optional): if determine the var or randomly. Defaults to None.

    Returns:
        Callable[[],torch.Tensor]: callable function that returns a tensor of gaussian distributed numbers
    """
    mean = mean if mean else torch.rand(1) * torch.randint(-5, 5, [1])
    var = var if var else torch.rand(1) * torch.randint(0, 10, [1])
    return lambda: (torch.randn(batch_size) * var + mean).unsqueeze(1)

In [25]:
f1, f2 = gaussians(mean=10.0, var=-5.0), gaussians(mean=-10.0, var=10.0)
ff.create_distplot(
    [f1().squeeze().tolist(), f2().squeeze().tolist()],
    ["1", "2"],
    show_rug=False,
).show()

In [58]:
# Create the models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 64
noise_dim = 100
output_dim = 1
loop_times = 1000
generator = Generator(noise_dim, output_dim).to(device)
discriminator = Discriminator(output_dim).to(device)
sampler = gaussians(batch_size)
# Define loss functions and optimizers
loss_function_D = nn.BCELoss()
loss_function_G = nn.BCELoss()
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.002)
optimizer_G = torch.optim.Adam(generator.parameters(), lr=0.002)

In [52]:
y = generator(torch.rand((10, noise_dim),device=device))

In [49]:
y.shape

torch.Size([10, 1])

In [56]:
discriminator(y)

tensor([[0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333],
        [0.5333]], device='cuda:0', grad_fn=<SigmoidBackward0>)

In [59]:
record_data = pd.DataFrame(
    {
        "loop_times": int,
        "loss_g": pd.NA,
        "loss_d": pd.NA,
        "generate_example": torch.Tensor,
    },
    index=range(loop_times),
)
# Training loop
for epoch in range(loop_times):
    # Sample real data and noise
    real_data = sampler().to(device)
    noise = torch.randn(batch_size, noise_dim).to(device)

    # Train discriminator
    real_labels = torch.ones((batch_size, 1)).to(device)
    fake_labels = torch.zeros((batch_size, 1)).to(device)

    # Real data pass
    discriminator_output = discriminator(real_data.float())
    D_real_loss = loss_function_D(discriminator_output, real_labels)

    # Fake data pass
    fake_data = generator(noise)
    discriminator_output = discriminator(fake_data)
    D_fake_loss = loss_function_D(discriminator_output, fake_labels)

    # Total discriminator loss
    D_loss = D_real_loss + D_fake_loss

    # Reset and update discriminator
    optimizer_D.zero_grad()
    D_loss.backward()
    optimizer_D.step()

    # Train generator
    fake_labels = torch.ones((batch_size, 1)).to(device)
    generator_output = generator(noise)
    G_loss = loss_function_G(discriminator(generator_output), fake_labels)

    # Reset and update generator
    optimizer_G.zero_grad()
    G_loss.backward()
    optimizer_G.step()

    record_data.loc[epoch] = [
        epoch,
        G_loss.item(),
        D_loss.item(),
        generator_output.cpu().detach().view(batch_size).numpy(),
    ]
    # Print losses and visualize progress (optional)
    if epoch % 100 == 0:
        print("Epoch:", epoch, "D_loss:", D_loss.item(), "G_loss:", G_loss.item())

Epoch: 0 D_loss: 1.4313173294067383 G_loss: 0.7686949372291565
Epoch: 100 D_loss: 1.3605389595031738 G_loss: 0.7337353825569153
Epoch: 200 D_loss: 1.3887770175933838 G_loss: 0.6904675960540771
Epoch: 300 D_loss: 1.3877086639404297 G_loss: 0.692396879196167
Epoch: 400 D_loss: 1.3876910209655762 G_loss: 0.6924910545349121
Epoch: 500 D_loss: 1.3874423503875732 G_loss: 0.6925970315933228
Epoch: 600 D_loss: 1.387584924697876 G_loss: 0.6923590302467346
Epoch: 700 D_loss: 1.3873252868652344 G_loss: 0.692612886428833
Epoch: 800 D_loss: 1.3872191905975342 G_loss: 0.6927001476287842
Epoch: 900 D_loss: 1.3871393203735352 G_loss: 0.6927716732025146


In [74]:
# @title Results { run: "auto" }
len_record_data = len(record_data)
index = 814  # @param {type: "slider", min: 0, max: 999}

ff.create_distplot(
    [
        sampler().view((batch_size)).tolist(),
        record_data["generate_example"][index].tolist(),
    ],
    ["rael", "generator"],
    show_rug=False,
    show_hist=False,
).show()
print(f"loss D: {record_data['loss_d'][index]}, loss G: {record_data['loss_g'][index]}")

loss D: 1.387110948562622, loss G: 0.6927058100700378


## GAN on mnist
try gan on mnist data|

In [ ]:
class Generator(nn.Module):
    def __init__(self, letant_dim: int, im_dim):
        super().__init__()
        self.im_dim = im_dim
        self.len_im = im_dim[0] * im_dim[1]
        self.letant_dim = letant_dim
        self.model = nn.Sequential(
            nn.Linear(letant_dim, 256),
            nn.LeakyReLU(),
            nn.Linear(256, self.len_im),
            nn.Tanh(),
        )

    def forward(self, X):
        return self.model(X.view((-1, self.letant_dim))).view((-1, *self.im_dim))


class Discrimnator(nn.Module):
    def __init__(self, im_dim) -> None:
        super().__init__()
        self.im_dim = im_dim
        self.len_im = im_dim[0] * im_dim[1]
        self.model = nn.Sequential(
            nn.Linear(self.len_im, 128), nn.LeakyReLU(), nn.Linear(128, 1), nn.Sigmoid()
        )

    def forward(self, X):
        return self.model(X.view(-1, self.len_im)).view(-1)

In [ ]:
transfomer = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0.5, 0.5)])
mnist_data = datasets.MNIST("/dataset", download=True, transform=transfomer)

In [ ]:
x = torch.stack([i[0] for i in mnist_data_se], dim=0)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
letant_dim = 100
im_dim = (28, 28)
generator = Generator(letant_dim, im_dim).to(device)
discrimnator = Discrimnator(im_dim).to(device)

In [ ]:
lr = 3e-4
optim_g = optim.Adam(generator.parameters(), lr=lr)
optim_d = optim.Adam(discrimnator.parameters(), lr=lr)
loss_f = nn.BCELoss()

In [ ]:
from math import ceil

num_epochs = 32
batch_size = 64
out_data = pd.DataFrame(
    {"epoch": pd.NA, "batch": pd.NA, "loss_g": pd.NA, "loss_d": pd.NA},
    index=range(num_epochs * ceil(len(x) / batch_size)),
)

In [ ]:
ind_out_data = 0
for epoch_ind in range(num_epochs):
    loader = dataloader.DataLoader(x, batch_size=batch_size, shuffle=True)
    for i_batch, X in enumerate(loader):
        optim_g.zero_grad()
        optim_d.zero_grad()
        noise = torch.randn(X.shape[0], letant_dim).to(device)
        fake = generator(noise)
        loss_d_fake = loss_f(
            discrimnator(fake), torch.zeros(fake.shape[0], device=device)
        )
        loss_d_real = loss_f(
            discrimnator(X.to(device)), torch.ones(X.shape[0], device=device)
        )
        loss_d = (loss_d_fake + loss_d_real) / 2
        loss_d.backward(retain_graph=True)
        optim_d.step()

        loss_g = loss_f(discrimnator(fake), torch.ones(fake.shape[0], device=device))
        loss_g.backward()
        optim_g.step()
        out_data.loc[ind_out_data] = [epoch_ind, i_batch, loss_g.item(), loss_d.item()]
        ind_out_data += 1
    with torch.no_grad():
        noise = torch.randn(10, letant_dim)
        im = generator(noise.to(device))
        clear_output(wait=True)
        px.imshow(im.cpu().detach(), facet_col=0, facet_col_wrap=5).show()
        px.line(out_data, x="batch", y="loss_g", color="epoch").show()